In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import numpy as np
import scipy.spatial
import pandas as pd
import sklearn.decomposition
import matplotlib.pyplot as plt
# import keras
from sklearn import preprocessing
from sklearn.decomposition import PCA
# %matplotlib inline
# Dimension reduction and clustering libraries
# import umap
# import hdbscan
import sys
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score
from sklearn.model_selection import LeaveOneOut,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import pairwise_distances,mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.insert(0, '../utils/') 
from readProfiles import *
from pred_models import *
from saveAsNewSheetToExistingFile import saveAsNewSheetToExistingFile

# from utils import networksEvol, tsne, readProfiles
# import umap

In [ ]:
pd.__version__

#### In this notebook we test how combinting two data modalities will improve preformance of the following tasks:
    1 - MoA classification
    2 - MoA clsutering

## MoA Prediction:
- Data Fusion Methods:
    - Early Fusion: simple concatenation
    - Late Fusion: desision probabilty average
    - RGCCA

### Paths

In [ ]:
procProf_dir='/home/ubuntu/bucket/projects/2018_04_20_Rosetta/workspace/'
results_dir='../results/'

### Read GE and CP profiles 

In [ ]:
################################################
# dataset options: 'CDRP' , 'LUAD', 'TAORF', 'LINCS', 'CDRP-bio'
dataset='LINCS'

################################################
# CP Profile Type options: 'augmented' , 'normalized', 'normalized_variable_selected'
profileType='normalized_variable_selected'
profTypeAbbrev=''.join([s[0] for s in profileType.split('_')])

################################################
# filtering to compounds which have high replicates for both GE and CP datasets
# highRepOverlapEnabled=0
# 'highRepUnion','highRepOverlap'
filter_perts='highRepUnion'
repCorrFilePath='./results/RepCor/RepCorrDF.xlsx'

filter_repCorr_params=[filter_perts,repCorrFilePath]

################################################
pertColName='PERT'

if filter_perts:
    f='filt'
else:
    f=''

mergProf_treatLevel,cp_features,l1k_features = \
read_paired_treatment_level_profiles(procProf_dir,dataset,profileType,filter_repCorr_params,1)

moa_col='Metadata_MoA'

##################################
if dataset=='LINCS':
    mergProf_treatLevel[moa_col]=mergProf_treatLevel['Metadata_moa']
    mergProf_treatLevel.loc[mergProf_treatLevel['Metadata_moa'].isnull(),moa_col]=\
    mergProf_treatLevel.loc[mergProf_treatLevel['Metadata_moa'].isnull(),'moa'].str.lower()
    mergProf_treatLevel['Compounds']=mergProf_treatLevel['PERT'].str[0:13]
    
elif dataset=='CDRP-bio':
    mergProf_treatLevel[moa_col]=mergProf_treatLevel['Metadata_moa'].str.lower()
    mergProf_treatLevel['Compounds']=mergProf_treatLevel['PERT'].str[0:13]

# mergProf_repLevel,mergProf_treatLevel,l1k_features,cp_features,pertColName=readMergedProfiles(dataset,profileType,nRep)
# cp_features,l1k_features=cp_features.tolist(),l1k_features.tolist()
# mergProf_repLevel['Compounds']=mergProf_repLevel['PERT'].str[0:13]

# if profileLevel=='replicate':
#     l1k=mergProf_repLevel[[pertColName]+l1k_features]
#     cp=mergProf_repLevel[[pertColName]+cp_features]
# elif profileLevel=='treatment':
l1k=mergProf_treatLevel[[pertColName,'Compounds',moa_col]+l1k_features]
cp=mergProf_treatLevel[[pertColName,'Compounds',moa_col]+cp_features]


scaler_ge = preprocessing.StandardScaler()
scaler_cp = preprocessing.StandardScaler()
l1k_scaled=l1k.copy()
l1k_scaled[l1k_features] = scaler_ge.fit_transform(l1k[l1k_features].values)
cp_scaled=cp.copy()
cp_scaled[cp_features] = scaler_cp.fit_transform(cp[cp_features].values.astype('float64'))


if 1:
    cp_scaled[cp_features] =preprocessing.MinMaxScaler(feature_range=(0, 1)).fit_transform(cp_scaled[cp_features].values)   
    l1k_scaled[l1k_features] =preprocessing.MinMaxScaler(feature_range=(0, 1)).fit_transform(l1k_scaled[l1k_features].values)           

    
merged_scaled=pd.concat([cp_scaled, l1k_scaled], axis=1)
merged_scaled = merged_scaled.loc[:,~merged_scaled.columns.duplicated()]    
merged_scaled['Compounds']=merged_scaled['PERT'].str[0:13]



#################### keep MOAs with more than "nSamplesMOA" compounds in their class

nSamplesMOA=4

nSamplesforEachMOAclass=mergProf_treatLevel.groupby(['Compounds']).sample(1).groupby([moa_col]).size().\
reset_index().rename(columns={0:'size'}).sort_values(by=['size'],ascending=False).reset_index(drop=True)


nSamplesforEachMOAclass2=mergProf_treatLevel.groupby([moa_col]).size().reset_index().rename(columns={0:'size'}).sort_values(by=['size'],ascending=False).reset_index(drop=True)

listOfSelectedMoAs=nSamplesforEachMOAclass[nSamplesforEachMOAclass['size']>nSamplesMOA][moa_col].tolist()
print('If we filter to MoAs which have more than',nSamplesMOA+1,' compounds in their category, ',\
      len(listOfSelectedMoAs),' out of ',nSamplesforEachMOAclass.shape[0] ,' MoAs remain.')

multi_label_MoAs=[l for l in listOfSelectedMoAs if '|' in l]
print('There are ',len(listOfSelectedMoAs),'MoA categories, which out of them ',len(multi_label_MoAs),\
      ' have multi labels and is removed')

listOfSelectedMoAs=[ele for ele in listOfSelectedMoAs if ele not in multi_label_MoAs]

le = preprocessing.LabelEncoder()
le.fit(listOfSelectedMoAs)

# IDs4filteredMOAs=mergProf_treatLevel[mergProf_treatLevel[moa_col].isin(listOfSelectedMoAs)][pertColName].unique().tolist()

rgcca_factors_cp=pd.read_csv("../JDR/results_CP_GE_lincs/factors_rgcca_cp.txt",sep="\t").reset_index(drop=True)
rgcca_factors_ge=pd.read_csv("../JDR/results_CP_GE_lincs/factors_rgcca_ge.txt",sep="\t").reset_index(drop=True).add_suffix('_ge')
rgcca_concat=pd.concat([rgcca_factors_cp,rgcca_factors_ge],axis=1)

filteredMOAs=merged_scaled[merged_scaled[moa_col].isin(listOfSelectedMoAs)].reset_index(drop=True)
data4eval=[[cp_scaled[cp_scaled[moa_col].isin(listOfSelectedMoAs)].reset_index(drop=True),cp_features],\
           [l1k_scaled[l1k_scaled[moa_col].isin(listOfSelectedMoAs)].reset_index(drop=True),l1k_features],\
           [filteredMOAs,cp_features+l1k_features],\
           [rgcca_factors_cp,rgcca_factors_cp.columns.tolist()],\
           [rgcca_factors_ge,rgcca_factors_ge.columns.tolist()],\
           [rgcca_concat,rgcca_concat.columns.tolist()]]

filteredMOAs['Metadata_moa_num']=le.transform(filteredMOAs[moa_col].tolist())

# nSamplesMOA=0 and union
print("There are ", filteredMOAs.shape[0],"samples across different doses of ",filteredMOAs['Compounds'].unique().shape[0] ,\
      "compounds", ", for ",filteredMOAs["Metadata_MoA"].unique().shape[0], "MoAs")

In [ ]:
# filteredMOAs.groupby(['Metadata_MoA']).size()
filteredMOAs.groupby(['Metadata_MoA','Compounds']).size().max()

In [ ]:
ls ../JDR/results_CP_GE_lincs/

In [ ]:
# merged_scaled['Compounds'].unique().shape

In [ ]:
# l1k_scaled[l1k_scaled[moa_col].isin(listOfSelectedMoAs)].reset_index(drop=True)

### LINCS Filtered sets

* Overlap:   
    - LINCS: Replicate Level Shapes (nSamples x nFeatures): cp:  52223 , 119 ,  l1k:  27837 , 978
    - l1k n of rep:  3.0
    - cp n of rep:  5.0
    - CP: from  9394  to  4647
    - l1k: from  8369  to  2338
    - CP and l1k high rep overlap:  1140
    - Treatment Level Shapes (nSamples x nFeatures+metadata): (1141, 122) (1141, 980) Merged Profiles Shape: (1141, 1101)
    - If we filter to MoAs which have more than 4  compounds in their category,  20  out of  284  MoAs remain.
    - There are  20 MoA categories, which out of them  0  have multi labels
    - There are  421 samples across different doses of  149 compounds , for  20 MoAs
        
* Union:
    - LINCS: Replicate Level Shapes (nSamples x nFeatures): cp:  52223 , 119 ,  l1k:  27837 , 978
    - l1k n of rep:  3.0
    - cp n of rep:  5.0
    - CP: from  9394  to  4647
    - l1k: from  8369  to  2338
    - CP and l1k high rep union:  5845
    - Treatment Level Shapes (nSamples x nFeatures+metadata): (5243, 122) (4431, 980) Merged Profiles Shape: (3828, 1101)
    - If we filter to MoAs which have more than 4  compounds in their category,  58  out of  514  MoAs remain.
    - There are  58 MoA categories, which out of them  1  have multi labels and is removed.
    - There are  1655 samples across different doses of  521 compounds , for  57 MoAs

#### Number of Classes with more than one label

In [ ]:
unq=mergProf_treatLevel['Metadata_MoA'].unique().tolist()
# n_multi_l=[len(u.split('|')) for u in unq if '|' in u]

fig, axes = plt.subplots(1,2,figsize=(10,5))

axes[0].hist([len(u.split('|')) for u in unq]);
axes[0].set_title('n>0');
axes[0].set_ylabel('number of multilabel classes - LINCS-filtSet-union');

axes[1].hist([len(u.split('|')) for u in unq if '|' in u]);
axes[1].set_title('n>1');#axes[1].set_ylabel('number of multilabel classes');

#### Number of Samples with more than one label

In [ ]:
unq=mergProf_treatLevel['Metadata_MoA'].tolist()
# n_multi_l=[len(u.split('|')) for u in unq if '|' in u]

fig, axes = plt.subplots(1,2,figsize=(10,5))

axes[0].hist([len(u.split('|')) for u in unq]);
axes[0].set_title('n>0');axes[0].set_ylabel('number of multilabel samples - LINCS-filtSet-union');

axes[1].hist([len(u.split('|')) for u in unq if '|' in u]);
axes[1].set_title('n>1');#axes[1].set_ylabel('number of multilabel samples');

In [ ]:
# plt.figure()
# fig, axes = plt.subplots(figsize=(20,5))
# nSamplesforEachMOAclass3=pd.merge(nSamplesforEachMOAclass,nSamplesforEachMOAclass2,how='inner',on=[moa_col])
nSamplesforEachMOAclass_=nSamplesforEachMOAclass[nSamplesforEachMOAclass['size']>1]
nSamplesforEachMOAclass2_=nSamplesforEachMOAclass2[nSamplesforEachMOAclass2[moa_col].isin(nSamplesforEachMOAclass_[moa_col].tolist())]
nSamplesforEachMOAclass_['h']='n-compounds'
nSamplesforEachMOAclass2_['h']='n-samples'

nSamplesforEachMOAclass3=pd.concat([nSamplesforEachMOAclass_,nSamplesforEachMOAclass2_])

sns.catplot(x="size",y=moa_col, data=nSamplesforEachMOAclass3[nSamplesforEachMOAclass3['size']>1],\
            orient="h", height=20, aspect=1,hue='h',
                kind="point", dodge=True, cut=0, bw=.2)
# fig.tight_layout()

### MoA prediction using each dataset and early and late fusion of datasets

#### Leave one out CV

In [ ]:
from sklearn.metrics import f1_score
from sklearn.utils import class_weight
from sklearn.naive_bayes import GaussianNB,ComplementNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict, GroupKFold,LeaveOneGroupOut
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import RandomOverSampler
# domXdata=merged_scaled.copy();
# domXfeats=data4eval[r][1]
# outdim_size=40

# filteredMOAs=domXdata[domXdata[pertColName].isin(IDs4filteredMOAs)].reset_index(drop=True)
# filteredMOAs['Metadata_moa_num']=le.transform(filteredMOAs[moa_col].tolist())

cls_model='mlp'#'mlp';#'lr'
moa_pred_res=pd.DataFrame(index=filteredMOAs['Compounds'].values,columns=['CP','GE','Early Fusion',\
                                    'RGCCA_CP','RGCCA_GE','RGCCA_EarlyFusion','Late Fusion','Metadata_moa_num'])

# parameter_space_MLP = {
#     'hidden_layer_sizes': [(50,),(100,),(200)],
#     'activation': ['tanh'],
#     'alpha': [0.0001],
# }
parameter_space_MLP = {
    'hidden_layer_sizes': [(50,),(100,),(200)]
}

parameter_space_logistic={"C":[500,1000,1500]}

from sklearn.model_selection import GridSearchCV



logo = LeaveOneGroupOut()

leG = preprocessing.LabelEncoder()
group_labels=leG.fit_transform(filteredMOAs['Compounds'].values)

i=0
# for train_index0, test_index in gkf.split(filteredMOAs, groups=group_labels):
for train_index, test_index in logo.split(filteredMOAs, groups=group_labels):
    print('rand ',i)
    i+=1
    
#     data_train = filteredMOAs.loc[train_index,domXfeats].values;
    labels_train=filteredMOAs.loc[train_index,'Metadata_moa_num'].tolist()
#     print(filteredMOAs.loc[train_index,'Metadata_moa_num'].unique().shape)
    
#     data_test = filteredMOAs.loc[test_index,domXfeats].values;
    labels_test=filteredMOAs.loc[test_index,'Metadata_moa_num'].tolist()
#     print(filteredMOAs.loc[test_index,'Metadata_moa_num'].unique().shape)    
    
    class_weightt = class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(labels_train),y=labels_train)    
#         model_tr = RandomForestClassifier(n_estimators=10,max_features=100,class_weight="balanced")

    overSampleMinorityFirstClassTo=np.max([labels_train.count(i) for i in set(labels_train)])
    
#     ratios = {l: overSampleMinorityFirstClassTo for l in set(labels_train) \
#       if labels_train.count(l)<overSampleMinorityFirstClassTo}
#     sm1=RandomOverSampler(ratio=ratios)
    sm1=RandomOverSampler(sampling_strategy='not majority',random_state=5)

    probs=[]
    
    for n,dt_modality,col in zip([0,1,2,3,4,5],data4eval,['CP','GE','Early Fusion','RGCCA_CP','RGCCA_GE','RGCCA_EarlyFusion']):

        
        data_m=dt_modality[0]

        dt_train=data_m.loc[train_index,dt_modality[1]].values;
        dt_test=data_m.loc[test_index,dt_modality[1]].values; 
    #         model_tr = RandomForestClassifier(n_estimators=10,max_features=100,class_weight="balanced")
#         model_tr = GaussianNB()
#         model_tr = ComplementNB()

#         model_tr = RandomForestClassifier(n_estimators=10)

        if cls_model=='lr':
#             model_tr = LogisticRegression(multi_class='multinomial',class_weight="balanced")        
        
            model_logistic = LogisticRegression(multi_class='multinomial',class_weight="balanced",n_jobs=4)        
            model_tr = GridSearchCV(model_logistic, parameter_space_logistic, n_jobs=4, cv=2)

        elif cls_model=='mlp':
            model_MLP = MLPClassifier(random_state=5,max_iter=100,alpha=0.0001,activation='tanh')
            model_tr = GridSearchCV(model_MLP, parameter_space_MLP, n_jobs=4, cv=2)
    
        dt_train_balanced,labels_train_balanced = sm1.fit_resample(dt_train,labels_train)
        
        model_tr.fit(dt_train_balanced,labels_train_balanced)
#         print(model_tr.best_params_)
#         accc=model_tr.score(dt_test,labels_test)
#         print(model_tr.predict(dt_test))
#         accc=f1_score(labels_test,model_tr.predict(dt_test), average='weighted')        
        moa_pred_res.loc[filteredMOAs.loc[test_index,'Compounds'].unique()[0],col]=model_tr.predict(dt_test)
#         moa_pred_res.loc[filteredMOAs.loc[test_index,'Compounds'].unique()[0],col]=accc*100
        probs.append(model_tr.predict_proba(dt_test))

#     labels_lateFusion=list(np.argmax((probs[0]+probs[1])/2,axis=1))
    labels_lateFusion=model_tr.classes_[np.argmax((probs[0]+probs[1])/2,axis=1)]
    moa_pred_res.loc[filteredMOAs.loc[test_index,'Compounds'].unique()[0],'Late Fusion']=\
    labels_lateFusion
    moa_pred_res.loc[filteredMOAs.loc[test_index,'Compounds'].unique()[0],'Metadata_moa_num']=\
    labels_test
#     f1_score(labels_test,labels_lateFusion, average='weighted')*100
#     accuracy_score(labels_test,labels_lateFusion)*100

moa_pred_res['Metadata_moa_num']=moa_pred_res.Metadata_moa_num.apply(lambda x: int(x[0]) if type(x)==list else x)    
print(moa_pred_res.mean())


filename=results_dir+'/MoAprediction/pred_moa_LINCS.xlsx'
saveAsNewSheetToExistingFile(filename,moa_pred_res,'fC-'+dataset+'-'+profTypeAbbrev+'-'+f+'-preds-'+cls_model+'-ht-3')


#### StratifiedGroupKFold

In [ ]:
# filteredMOAs['Metadata_moa_num'].values
# filteredMOAs

In [ ]:
from sklearn.metrics import f1_score
from sklearn.utils import class_weight
from sklearn.naive_bayes import GaussianNB,ComplementNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict, GroupKFold,StratifiedGroupKFold
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import RandomOverSampler
# domXdata=merged_scaled.copy();
# domXfeats=data4eval[r][1]
# outdim_size=40

# filteredMOAs=domXdata[domXdata[pertColName].isin(IDs4filteredMOAs)].reset_index(drop=True)
# filteredMOAs['Metadata_moa_num']=le.transform(filteredMOAs[moa_col].tolist())

# cls_model='lr'#'mlp';#'lr'
for cls_model in ['mlp','lr']:
    moa_pred_res=pd.DataFrame(index=filteredMOAs.index,columns=['CP','GE','Early Fusion',\
                                        'RGCCA_CP','RGCCA_GE','RGCCA_EarlyFusion','Late Fusion','Metadata_moa_num'])


    moa_pred_res['PERT']=filteredMOAs['PERT']
    moa_pred_res['Compounds']=filteredMOAs['Compounds']

    # parameter_space_MLP = {
    #     'hidden_layer_sizes': [(50,),(100,),(200)],
    #     'activation': ['tanh'],
    #     'alpha': [0.0001],
    # }
    parameter_space_MLP = {
        'hidden_layer_sizes': [(100,),(200),(400)],
        'activation': ['tanh', 'relu'],
        'alpha': [0.0001,0.05],
        'learning_rate': ['constant','adaptive'],
    }

    parameter_space_logistic={"C":[500,1000,1500]}

    from sklearn.model_selection import GridSearchCV



    # logo = LeaveOneGroupOut()

    sgkf = StratifiedGroupKFold(n_splits=5,shuffle=True,random_state=2)

    leG = preprocessing.LabelEncoder()
    group_labels=leG.fit_transform(filteredMOAs['Compounds'].values)

    i=0
    # for train_index0, test_index in gkf.split(filteredMOAs, groups=group_labels):
    for train_index, test_index in sgkf.split(filteredMOAs,filteredMOAs['Metadata_moa_num'].values, groups=group_labels):
        print('rand ',i)
        i+=1

    #     data_train = filteredMOAs.loc[train_index,domXfeats].values;
        labels_train=filteredMOAs.loc[train_index,'Metadata_moa_num'].tolist()
    #     print(filteredMOAs.loc[train_index,'Metadata_moa_num'].unique().shape)

    #     data_test = filteredMOAs.loc[test_index,domXfeats].values;
        labels_test=filteredMOAs.loc[test_index,'Metadata_moa_num'].tolist()
#         print(filteredMOAs.loc[test_index,'Metadata_moa_num'].unique().shape)    
#         print(len(labels_test))
#         print(filteredMOAs.loc[test_index,:].groupby(['Metadata_moa_num','Compounds']).size())
    #     class_weightt = class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(labels_train),y=labels_train)    
    #         model_tr = RandomForestClassifier(n_estimators=10,max_features=100,class_weight="balanced")

    #     overSampleMinorityFirstClassTo=np.max([labels_train.count(i) for i in set(labels_train)])

    #     ratios = {l: overSampleMinorityFirstClassTo for l in set(labels_train) \
    #       if labels_train.count(l)<overSampleMinorityFirstClassTo}
    #     sm1=RandomOverSampler(ratio=ratios)
        moa_pred_res.loc[test_index,'Fold']=i

        sm1=RandomOverSampler(sampling_strategy='not majority',random_state=5)

        probs=[]

        for n,dt_modality,col in zip([0,1,2,3,4,5],data4eval,['CP','GE','Early Fusion','RGCCA_CP',\
                    'RGCCA_GE','RGCCA_EarlyFusion','Fold']):


            data_m=dt_modality[0]

            dt_train=data_m.loc[train_index,dt_modality[1]].values;
            dt_test=data_m.loc[test_index,dt_modality[1]].values; 
        #         model_tr = RandomForestClassifier(n_estimators=10,max_features=100,class_weight="balanced")
    #         model_tr = GaussianNB()
    #         model_tr = ComplementNB()

    #         model_tr = RandomForestClassifier(n_estimators=10)

            if cls_model=='lr':
    #             model_tr = LogisticRegression(multi_class='multinomial',class_weight="balanced")        

                model_logistic = LogisticRegression(multi_class='multinomial',class_weight="balanced",n_jobs=4)        
                model_tr = GridSearchCV(model_logistic, parameter_space_logistic, n_jobs=4, cv=3)

            elif cls_model=='mlp':
#                 model_MLP = MLPClassifier(random_state=5,max_iter=100,alpha=0.0001,activation='tanh')
                model_MLP = MLPClassifier(random_state=5,max_iter=500)
                model_tr = GridSearchCV(model_MLP, parameter_space_MLP, n_jobs=4, cv=3)

            dt_train_balanced,labels_train_balanced = sm1.fit_resample(dt_train,labels_train)

            model_tr.fit(dt_train_balanced,labels_train_balanced)
    #         print(model_tr.best_params_)
    #         accc=model_tr.score(dt_test,labels_test)
    #         print(model_tr.predict(dt_test))
    #         accc=f1_score(labels_test,model_tr.predict(dt_test), average='weighted')        
            moa_pred_res.loc[test_index,col]=model_tr.predict(dt_test)
    #         moa_pred_res.loc[filteredMOAs.loc[test_index,'Compounds'].unique()[0],col]=accc*100
            probs.append(model_tr.predict_proba(dt_test))

    #     labels_lateFusion=list(np.argmax((probs[0]+probs[1])/2,axis=1))
        labels_lateFusion=model_tr.classes_[np.argmax((probs[0]+probs[1])/2,axis=1)]
        moa_pred_res.loc[test_index,'Late Fusion']=\
        labels_lateFusion
        moa_pred_res.loc[test_index,'Metadata_moa_num']=\
        labels_test
    #     f1_score(labels_test,labels_lateFusion, average='weighted')*100
    #     accuracy_score(labels_test,labels_lateFusion)*100

    moa_pred_res['Metadata_moa_num']=moa_pred_res.Metadata_moa_num.apply(lambda x: int(x[0]) if type(x)==list else x)    
    print(moa_pred_res.mean())


    filename=results_dir+'/MoAprediction/pred_moa_LINCS.xlsx'
    saveAsNewSheetToExistingFile(filename,moa_pred_res,'fC-'+dataset+'-'+profTypeAbbrev+'-'+f+'-preds-'+cls_model+'-ht-sgkf-10f')


## Visualize the results

In [ ]:
filename=results_dir+'/MoAprediction/pred_moa.xlsx'
saved_scores=pd.read_excel(filename, sheet_name=None)
saved_scores.keys()

In [ ]:
filename=results_dir+'/MoAprediction/pred_moa_LINCS.xlsx'
saved_scores=pd.read_excel(filename, sheet_name=None)
saved_scores.keys()

In [ ]:
from sklearn.metrics import f1_score
res_df=pd.DataFrame(columns=['Modality','Model'])

scores_mlp=saved_scores['fC-LINCS-nvs-filt-preds-mlp-ht-sgkf-1']

for dd in ['CP', 'GE', 'Early Fusion', 'Late Fusion','RGCCA_EarlyFusion']:
    scor_vals=scores_mlp[dd].apply(lambda x: int(eval(x)[0]) if type(x)==str else x)    
    f1=f1_score(scor_vals,scores_mlp.Metadata_moa_num.values, average='weighted')
    temp_df = pd.DataFrame(data={'Modality': [dd], 'Model': ['MLP'],'F1-score':[f1*100]})
    res_df=res_df.append(temp_df)
    
    
scores_mlp=saved_scores['fC-LINCS-nvs-filt-preds-lr-ht-sgkf-1']

for dd in ['CP', 'GE', 'Early Fusion', 'Late Fusion','RGCCA_EarlyFusion']:
    scor_vals=scores_mlp[dd].apply(lambda x: int(eval(x)[0]) if type(x)==str else x)    
    f1=f1_score(scores_mlp.Metadata_moa_num.values,scor_vals, average='weighted')
    temp_df = pd.DataFrame(data={'Modality': [dd], 'Model': ['Logistic R'],'F1-score':[f1*100]})
    res_df=res_df.append(temp_df)
    

In [ ]:
# scor_vals

In [ ]:
# saveAsNewSheetToExistingFile(filename,saved_scores['fC-LINCS-nvs-filt-preds-mlp-ht'],\
#                              'fC-LINCS-nvs-filt-preds-mlp-ht2')
# ax.patches
# scor_vals
# scores_mlp[dd]=scores_mlp[dd].apply(lambda x: int(eval(x)[0]) if type(x)==str else x)    
# f1_score(scores_mlp[scores_mlp['Metadata_moa_num']==0].Metadata_moa_num.values,scores_mlp[scores_mlp['Metadata_moa_num']==0][dd], positiveaverage=None)
# f1_score(ytest, ypred_prob, pos_label=0)
# report = classification_report(y_test, y_pred, )
from sklearn.metrics import classification_report
scores_mlp=saved_scores['fC-LINCS-nvs-filt-preds-mlp-ht-sgkf-1']
scor_vals=scores_mlp['RGCCA_EarlyFusion'].apply(lambda x: int(eval(x)[0]) if type(x)==str else x)    
scor_vals_cp=scores_mlp['CP'].apply(lambda x: int(eval(x)[0]) if type(x)==str else x)    
scor_vals_ge=scores_mlp['GE'].apply(lambda x: int(eval(x)[0]) if type(x)==str else x)    

report_clss=pd.DataFrame(classification_report(scores_mlp.Metadata_moa_num.values,scor_vals.values,output_dict=True)).transpose()
report_clss_cp=pd.DataFrame(classification_report(scores_mlp.Metadata_moa_num.values,scor_vals_cp.values,output_dict=True)).transpose()
report_clss_ge=pd.DataFrame(classification_report(scores_mlp.Metadata_moa_num.values,scor_vals_ge.values,output_dict=True)).transpose()

# report_clss[list(range(13))]
# n_classes=14
n_classes=57
report_clss['MoA']=''
# report_clss
report_clss.loc[0:n_classes,'MoA']=le.inverse_transform(list(range(n_classes)))
report_clss2=report_clss[0:n_classes]

In [ ]:
# report_clss_cp.loc[report_clss3.index]['f1-score']
report_clss4[0:10]

In [ ]:
%matplotlib inline
import matplotlib.style as style
style.use('seaborn-colorblind')
sns.set_context("paper",font_scale = 1.5, rc={"font.size":4,"axes.titlesize":14,"axes.labelsize":15})
sns.set_style("whitegrid")

report_clss3=report_clss2.sort_values(by='f1-score',ascending=False)
report_clss3['f1-score-cp']=report_clss_cp.loc[report_clss3.index]['f1-score'].values
report_clss3['f1-score-ge']=report_clss_ge.loc[report_clss3.index]['f1-score'].values
report_clss4=report_clss3[['f1-score-cp','f1-score-ge','f1-score']]
report_clss4=report_clss4.rename(columns={'f1-score-cp':'CP','f1-score-ge':'GE','f1-score':'RGCCA_EarlyFusion'})
report_clss4.index=report_clss3['MoA']
ax = report_clss4.plot.bar(rot=90,figsize=(12,6))
ax.set_ylim([0,1])
plt.title('MoA class specific F1-scores')
plt.tight_layout()

# ax = report_clss4[:15].plot.bar(rot=90,figsize=(7,7))
ax =report_clss4[report_clss4["RGCCA_EarlyFusion"]>0].plot.bar(rot=90,figsize=(12,6))
ax.set_ylim([0,1])
plt.title('MoA class specific F1-scores\n LINCS')
plt.tight_layout()
# plt.savefig('moa_class_spacific_bar.eps')

if 1:
    source_data_add='../Figs_Source_Data.xlsx'
    saveAsNewSheetToExistingFile(source_data_add,report_clss4,'fig4c_2')

    # fig.tight_layout()
    plt.savefig('fig4c_2.pdf',papertype='letter',dpi=500)
    

In [ ]:
# report_clss4[report_clss4["RGCCA_EarlyFusion"]==0].shape

In [ ]:
# report_clss4[report_clss4["RGCCA_EarlyFusion"]>0].index

In [ ]:
lincs_moa_top=['mTOR inhibitor', 'glucocorticoid receptor agonist', 'HSP inhibitor',
       'proteasome inhibitor', 'MEK inhibitor', 'HDAC inhibitor',
       'tubulin polymerization inhibitor', 'PLK inhibitor', 'EGFR inhibitor',
       'retinoid receptor agonist', 'CDK inhibitor', 'topoisomerase inhibitor',
       'p38 MAPK inhibitor', 'AKT inhibitor', 'serotonin receptor antagonist',
       'gamma secretase inhibitor', 'HMGCR inhibitor', 'PI3K inhibitor',
       'rho associated kinase inhibitor', 'cyclooxygenase inhibitor',
       'protein synthesis inhibitor', 'histamine receptor antagonist',
       'sterol demethylase inhibitor', 'PKC inhibitor',
       'dopamine receptor antagonist', 'adrenergic receptor antagonist',
       'vitamin D receptor agonist', 'acetylcholine receptor antagonist',
       'progesterone receptor agonist', 'dopamine receptor agonist',
       'benzodiazepine receptor agonist', 'phosphodiesterase inhibitor',
       'calcium channel blocker', 'adrenergic receptor agonist']

cdrp_moa_top=['tubulin polymerization inhibitor', 'atpase inhibitor',
       'glucocorticoid receptor agonist', 'cdk inhibitor',
       'adrenergic receptor agonist', 'dopamine receptor antagonist',
       'protein synthesis inhibitor', 'serotonin receptor antagonist',
       'calcium channel blocker', 'dopamine receptor agonist',
       'serotonin receptor agonist', 'adrenergic receptor antagonist']

top_cdrp_lincs_overlap=list(set(lincs_moa_top) & set(cdrp_moa_top))

In [ ]:
top_cdrp_lincs_overlap

In [ ]:
# ax = report_clss4[:15].plot.bar(rot=90,figsize=(7,7))
ax =report_clss4.loc[top_cdrp_lincs_overlap].plot.bar(rot=90,figsize=(6,6))
ax.set_ylim([0,1])
plt.title('MoA class specific F1-scores')
plt.tight_layout()